In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [ ]:
def f(x,y):
    return x % 1.0

In [ ]:
def visualise(f, minx=0.0, maxx=1.0, miny=0.0, maxy=1.0, width=256, height=256):
    xwidth = maxx - minx
    yheight = maxy - miny
    
    def get_pixel(row, column):
        x = column / (width-1) * xwidth + minx
        y = row / (height-1) * yheight + miny
        return f(x,y)

    pixels = np.flipud(np.fromfunction(get_pixel, (height, width))).astype(float)*255
    return Image.fromarray(pixels).convert('L')

In [ ]:
visualise(f)

In [ ]:
visualise(lambda x,y:x+y)

In [ ]:
columns = 20
rows = 20

x = np.arange(0,columns,1)
y = np.arange(0,rows,1)

X, Y = np.meshgrid(x, y)

SEED = [1,2,3,4]
random = np.random.RandomState(SEED)

random_angles = random.rand(rows,columns) * (2 * np.pi)

x_components = np.cos(random_angles)
y_components = np.sin(random_angles)


In [ ]:
fig, ax = plt.subplots()

x_pos = X
y_pos = Y
x_direct = x_components
y_direct = y_components

assert x_pos.shape == y_pos.shape == x_direct.shape == y_direct.shape

ax.quiver(x_pos,y_pos,x_direct,y_direct, angles='xy', scale_units='xy', scale=2)

plt.show()

In [ ]:

def colour_point_by_gradient(x, y, gradient_column, gradient_row):
    gx = x_components[gradient_row, gradient_column]
    gy = y_components[gradient_row, gradient_column]
    dot_product = (x * gx) + (y * gy)

    return dot_product * 0.5 + 0.5

def f(x,y):
    return colour_point_by_gradient(x,y,3,5)

visualise(f, minx=-1.0, maxx=1.0, miny=-1.0, maxy=1.0)

In [ ]:
COLOUR_FUNC = Callable[[float, float], float]
GRID_FUNC = Callable[[int, int], COLOUR_FUNC]

def interpolate_nearest_neighbour(get_colour_func_for_grid_cell: GRID_FUNC) -> COLOUR_FUNC:
    def get_pixel_colour(x: float,y: float) -> float:
        grid_x = np.round(x).astype(int)
        grid_y = np.round(y).astype(int)

        pixel_func = get_colour_func_for_grid_cell(grid_x,grid_y)
        return pixel_func(x,y)
    
    return get_pixel_colour

def get_gradient_colour_func_for_grid_cell(grid_x: int, grid_y: int) -> COLOUR_FUNC:
    def get_pixel_colour(x,y):
        return colour_point_by_gradient(x - grid_x, y - grid_y, grid_x, grid_y)

    return get_pixel_colour

visualise(interpolate_nearest_neighbour(get_gradient_colour_func_for_grid_cell), minx=0, maxx=5.0, miny=0, maxy=5.0)

In [ ]:
def lerp(start, end, t):
    return start * (1 - t) + t * end
def smoothstep(start, end, t):
    return start + t * t * (3.0 - 2.0 * t) * (end - start)
def nearest_neighbour(start, end, t):
    if t>=0.5:
        return end
    return start

In [ ]:
def interpolate_linearly(get_colour_func_for_grid_cell: GRID_FUNC) -> COLOUR_FUNC:
    def get_pixel_colour(x: float,y: float) -> float:
        left_grid_x = np.floor(x).astype(int)
        right_grid_x = left_grid_x + 1
        lower_grid_y = np.floor(y).astype(int)
        upper_grid_y = lower_grid_y + 1

        x_remainder = x - left_grid_x
        y_remainder = y - lower_grid_y

        lower_left_pixel = get_colour_func_for_grid_cell(left_grid_x, lower_grid_y)(x,y)
        lower_right_pixel = get_colour_func_for_grid_cell(right_grid_x, lower_grid_y)(x,y)
        upper_left_pixel = get_colour_func_for_grid_cell(left_grid_x, upper_grid_y)(x,y)
        upper_right_pixel = get_colour_func_for_grid_cell(right_grid_x, upper_grid_y)(x,y)

        
        lower_lerp_pixel = smoothstep(lower_left_pixel, lower_right_pixel, x_remainder)
        upper_lerp_pixel = smoothstep(upper_left_pixel, upper_right_pixel, x_remainder)

        final_pixel = smoothstep(lower_lerp_pixel, upper_lerp_pixel, y_remainder)
        return final_pixel
    
    return get_pixel_colour

visualise(interpolate_linearly(get_gradient_colour_func_for_grid_cell), minx=0, maxx=18.9, miny=0, maxy=18.9)